In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin
import re


In [3]:
url = "https://www.blom.com.jo/retail-jordan/home"

base_url = "https://www.blom.com.jo"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")


In [5]:
folder_name = "blom_banner_images"
os.makedirs(folder_name, exist_ok=True)

In [7]:
images = []
bank_name = "BLOM Bank Jordan"

In [9]:
banner_section = soup.find("div", id="homeBannersHolder")
if banner_section:
    # كل li داخل slides
    slides = banner_section.find_all("li")

    for slide in slides:
        img_tag = slide.find("img")
        image_link = ""

        # 1. إذا فيه <img> طبيعي
        if img_tag and img_tag.get("src"):
            image_link = img_tag.get("src").strip()

        # 2. إذا مافي، دور على background-image داخل figure
        if not image_link:
            figure = slide.find("figure")
            if figure and figure.has_attr("style"):
                style = figure["style"]
                match = re.search(r"url\(['\"]?(.*?)['\"]?\)", style)
                if match:
                    image_link = match.group(1)

        if image_link:
            # تكملة الرابط لو ناقص
            if not image_link.startswith("http"):
                full_img_url = urljoin(base_url, image_link)
            else:
                full_img_url = image_link

            # استخراج اسم الصورة
            img_name = image_link.split("/")[-1].split("?")[0]

            # مسار حفظ الصورة
            img_path = os.path.join(folder_name, img_name)

            # تحميل الصورة
            try:
                img_response = requests.get(full_img_url)
                img_response.raise_for_status()
                with open(img_path, "wb") as f:
                    f.write(img_response.content)
                print(f"✔️ تم تحميل الصورة: {img_name}")

                # إضافة للـ Data
                images.append({
                    "Bank_Name": bank_name,
                    "Image_Name": img_name,
                    "Image_Link": full_img_url,
                    "Article_Link": url  # كلهم بيرجعوا للصفحة الرئيسية بهالحالة
                })
            except Exception as e:
                print(f"❌ فشل تحميل الصورة: {full_img_url}, السبب: {e}")

else:
    print("❌ لم يتم العثور على قسم البانر!")


❌ لم يتم العثور على قسم البانر!


In [13]:
df = pd.DataFrame(images)
df

""
